In [ ]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 5.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import random
import warnings
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, precision_score, accuracy_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neural_network import MLPClassifier
from deap import base, creator, tools, algorithms

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_parquet('enade_cenario_1_2_faixas_modificadas.parquet.gzip')
df.head()

,REGIAO_IES,SEXO,COR_RACA,ESTADO_CIVIL,RENDA_FAMILIAR,ESCOLARIDADE_PAI,ESCOLARIDADE_MAE,COMPANHIA_RESIDENCIA,TIPO_ESCOLA_ENSINO_MEDIO,MODALIDADE_ENSINO_MEDIO,...,OFERTA_APOIO_GERAL,SITUACAO_TRABALHO,SITUACAO_FINANCEIRA,DEDICACAO_SEMANAL,GRAU_ACADEMICO_CURSO,REGIAO_ESCOLA_ENSINO_MEDIO,GRANDE_AREA_CURSO,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO,TEMPO_GRADUACAO_1,FAIXA_ETARIA
0,REGIAO_IES: Sul,SEXO: Masculino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: Até 1,5 salário mínimo (até R$...",ESCOLARIDADE_PAI: Nenhuma,ESCOLARIDADE_MAE: Nenhuma,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino médio tradicional,...,OFERTA_APOIO_GERAL: Não,SITUACAO_TRABALHO: Não,SITUACAO_FINANCEIRA: Não tenho renda e meus ga...,"DEDICACAO_SEMANAL: Nenhuma, apenas assisto às ...",GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Norte,GRANDE_AREA_CURSO: Serviços,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 2...,TEMPO_GRADUACAO: Entre 2 e 7 anos,FAIXA_ETARIA: Entre 18 e 24 anos
1,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: De 3 a 4,5 salários mínimos (R...",ESCOLARIDADE_PAI: Ensino médio,ESCOLARIDADE_MAE: Ensino superior,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino médio tradicional,...,OFERTA_APOIO_GERAL: Não,SITUACAO_TRABALHO: Sim,SITUACAO_FINANCEIRA: Tenho renda e contribuo p...,DEDICACAO_SEMANAL: De uma a três,GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Sul,GRANDE_AREA_CURSO: Serviços,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 2...,TEMPO_GRADUACAO: Entre 2 e 7 anos,FAIXA_ETARIA: Entre 25 e 39 anos
2,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: Até 1,5 salário mínimo (até R$...",ESCOLARIDADE_PAI: Ensino médio,ESCOLARIDADE_MAE: Ensino médio,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino médio tradicional,...,OFERTA_APOIO_GERAL: Sim,SITUACAO_TRABALHO: Não,"SITUACAO_FINANCEIRA: Tenho renda, mas recebo a...",DEDICACAO_SEMANAL: De quatro a sete,GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Sul,GRANDE_AREA_CURSO: Serviços,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: E...,TEMPO_GRADUACAO: Entre 2 e 7 anos,FAIXA_ETARIA: Entre 18 e 24 anos
3,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),"RENDA_FAMILIAR: Até 1,5 salário mínimo (até R$...",ESCOLARIDADE_PAI: Nenhuma,ESCOLARIDADE_MAE: Nenhuma,"COMPANHIA_RESIDENCIA: Em casa ou apartamento, ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino médio tradicional,...,OFERTA_APOIO_GERAL: Não,SITUACAO_TRABALHO: Não,SITUACAO_FINANCEIRA: Não tenho renda e meus ga...,"DEDICACAO_SEMANAL: Nenhuma, apenas assisto às ...",GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Norte,GRANDE_AREA_CURSO: Serviços,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: E...,TEMPO_GRADUACAO: Entre 2 e 7 anos,FAIXA_ETARIA: Entre 25 e 39 anos
4,REGIAO_IES: Sul,SEXO: Feminino,COR_RACA: Branca,ESTADO_CIVIL: Solteiro(a),RENDA_FAMILIAR: De 10 a 30 salários mínimos (R...,ESCOLARIDADE_PAI: Ensino superior,ESCOLARIDADE_MAE: Ensino superior,"COMPANHIA_RESIDENCIA: Em casa, apartamento ou ...",TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior part...,MODALIDADE_ENSINO_MEDIO: Ensino médio tradicional,...,OFERTA_APOIO_GERAL: Sim,SITUACAO_TRABALHO: Sim,"SITUACAO_FINANCEIRA: Tenho renda, mas recebo a...",DEDICACAO_SEMANAL: De uma a três,GRAU_ACADEMICO_CURSO: Bacharelado,REGIAO_ESCOLA_ENSINO_MEDIO: Sudeste,GRANDE_AREA_CURSO: Serviços,DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 2...,TEMPO_GRADUACAO: Entre 2 e 7 anos,FAIXA_ETARIA: Entre 18 e 24 anos


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse = False)
X = enc.fit_transform(df.drop('TEMPO_GRADUACAO_1', axis = 1))
y = df['TEMPO_GRADUACAO_1']

In [ ]:
rus = RandomUnderSampler(random_state=0)
X, y = rus.fit_resample(X, y)

In [ ]:
one_hot_encoded_frame =  pd.DataFrame(X, columns= enc.get_feature_names())

In [ ]:
model = MLPClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(one_hot_encoded_frame, y, test_size=0.3)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X_train

,x0_REGIAO_IES: Centro-Oeste,x0_REGIAO_IES: Nordeste,x0_REGIAO_IES: Norte,x0_REGIAO_IES: Sudeste,x0_REGIAO_IES: Sul,x1_SEXO: Feminino,x1_SEXO: Masculino,x2_COR_RACA: Amarela,x2_COR_RACA: Branca,x2_COR_RACA: Indígena,...,x26_GRANDE_AREA_CURSO: Saúde e bem-estar,x26_GRANDE_AREA_CURSO: Serviços,x27_DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 10 anos ou mais,x27_DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: 2 anos ou menos,x27_DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: Entre 3 e 5 anos,x27_DIFERENCA_INICIO_GRADUACAO_FIM_ENSINO_MEDIO: Entre 6 e 9 anos,x28_FAIXA_ETARIA: 60 anos ou mais,x28_FAIXA_ETARIA: Entre 18 e 24 anos,x28_FAIXA_ETARIA: Entre 25 e 39 anos,x28_FAIXA_ETARIA: Entre 40 e 59 anos
2600,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
11970,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6129,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2127,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
12055,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3577,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
6077,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
13512,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
clf_full = model.fit(X_train,y_train)
y_pred = clf_full.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.7723975335509612


In [ ]:
# Create a fitness function with weight 1. 
# If it's a maximization function the weight should be positive else,for minimization, it should be negative 
# Here, we want to maximize accuracy
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Create blank individual as list with the fitness function defined above.
# Individual is synonymous with chromosome
creator.create("Individual", list, fitness=creator.FitnessMax)
# Declare the size of the chromosome. Here the size will be no. of variables
ind_size = X_train.shape[1]
# Declare the toolbox. Toolbox is a container which contains all the operators declared
# To include any operator in the toolbox, it has to registered
toolbox = base.Toolbox()
# Register a random number generator function to randomly generate 1 or 0.
# This will signify whether a variable is present or not in a particular individual. 1: Present; 0: Absent
# 1/0 values, here, are alleles
toolbox.register("attrib_bin", random.randint, 0, 1)
# Insert values in individuals which will have value from random number generator defined above
# size of each individual will be no. of variables
# Register the individuals in the toolbox
# initRepeat argument is used to initialize the individual
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attrib_bin, n=ind_size)
# Define a population with the individuals created above
# Register the population in toolbox
# initRepeat argument is used to initialize the population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# ind1 = toolbox.individual()
print("Total no. of variables/length of each chromosome = ", ind_size)

Total no. of variables/length of each chromosome =  116


In [ ]:
def evaluate(individual):
    sum_features = np.sum(individual)
    if sum_features == 0:
        return 0.0
    else:
        sum_features = np.sum(individual)
        collector_train = []
        collector_test = []
        for k in range(0, len(individual)):
            if individual[k] == 1:
                collector_train.append(X_train.iloc[:, k])
                collector_test.append(X_test.iloc[:, k])
        X_train1 = np.transpose(np.array(collector_train))
        X_test1 = np.transpose(np.array(collector_test))
        np.nan_to_num(X_train1, copy=False)
        np.nan_to_num(y_train, copy=False)
        np.nan_to_num(X_test1, copy=False)
        np.nan_to_num(y_test, copy=False)
        clf = model.fit(X_train1, y_train)
        y_pred = clf.predict(X_test1)
        # accuracy = (y_test['target'].tolist()==y_pred).sum() / y_test['target'].count()
        accuracy = accuracy_score(y_test, y_pred)
        return (accuracy,)

In [ ]:
# Two points crossover
toolbox.register("mate", tools.cxTwoPoint)
# Bit flip mutation The indpb argument is the probability of each attribute to be flipped
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
# Select the best individual among tournsize randomly chosen individuals
toolbox.register("select", tools.selTournament, tournsize=3)
# Register the fitness function defined above in the toolbox
toolbox.register("evaluate", evaluate)
# Define the statistics to be shown during the algorithm run.
# We have selected minimum, maximum and average accuracy for each generation of run
# Decision will, however, be taken based on maximum accuracy as defined earlier
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Min", np.min)

In [ ]:
#Define initial population with its size. Here we are defining low population size (only 20) for demonstration
pop = toolbox.population(n=20)
# Below HallOfFame argument saves k best individual across all the generations
# We will use this to determine our final variable combination. 
# We are taking only the first best one. It is possible to save k number of best solution in hof
hof = tools.HallOfFame(1)
# Now, let's run the algorithm with the most simple in-built method available in DEAP (eaSimple).
# The arguments are as below:
# pop: Population defined earlier
# toolbox: toolbox containing all the operator defined
# cxpb: The probability of mating two individuals.
# mutpb: The probability of mutating an individual. We are keeping it high to show the impact
# ngen: The number of generation.
pop, log = algorithms.eaSimple(
    pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=30, halloffame=hof, stats=stats, verbose=True)
# Get the best individual
best = hof.items[0]
#Print the best individual and it's accuracy
print("Best Individual = ", best)
print("Best Fitness = ", best.fitness.values[0])

gen	nevals	Mean    	Max     	Min     
0  	20    	0.736942	0.789808	0.631665
1  	14    	0.770158	0.785092	0.746826
2  	14    	0.777258	0.785092	0.765325
3  	14    	0.778808	0.792347	0.761697
4  	14    	0.782417	0.792347	0.772942
5  	17    	0.781538	0.792347	0.762967
6  	14    	0.783487	0.79144 	0.764599
7  	15    	0.786063	0.796337	0.770947
8  	13    	0.786126	0.796337	0.775118
9  	16    	0.7859  	0.795793	0.765506
10 	13    	0.788212	0.795974	0.772579
11 	9     	0.790325	0.797606	0.778564
12 	19    	0.791567	0.801052	0.780015
13 	15    	0.791794	0.803772	0.77947 
14 	16    	0.792691	0.803772	0.776931
15 	11    	0.794324	0.803772	0.782009
16 	13    	0.792537	0.803772	0.777476
17 	13    	0.790497	0.803772	0.780559
18 	10    	0.791277	0.803772	0.777113
19 	14    	0.791721	0.800145	0.779833
20 	13    	0.788964	0.800145	0.775118
21 	11    	0.792845	0.800145	0.779108
22 	14    	0.789635	0.800145	0.776931
23 	13    	0.793707	0.802503	0.778201
24 	18    	0.789645	0.804135	0.780015
25 	14    	0

In [ ]:
counter_1 = 0
for i in best:
  if i == 1:
    counter_1 += 1
print(counter_1)

59


In [ ]:
best_collector = []
for k in range(0, 116):
  if best[k] == 1:
    best_collector.append(X_train.columns[k])
print("No. of variables select = ", len(best_collector))
print(best_collector)

No. of variables select =  59
['x0_REGIAO_IES: Centro-Oeste', 'x0_REGIAO_IES: Nordeste', 'x0_REGIAO_IES: Norte', 'x0_REGIAO_IES: Sudeste', 'x0_REGIAO_IES: Sul', 'x1_SEXO: Masculino', 'x2_COR_RACA: Amarela', 'x2_COR_RACA: Branca', 'x3_ESTADO_CIVIL: Casado(a)', 'x3_ESTADO_CIVIL: Solteiro(a)', 'x4_RENDA_FAMILIAR: Acima de 30 salários mínimos (mais de R$ 28.620,00)', 'x4_RENDA_FAMILIAR: Até 1,5 salário mínimo (até R$ 1.431,00)', 'x4_RENDA_FAMILIAR: De 4,5 a 6 salários mínimos (R$ 4.293,01 a R$ 5.724,00)', 'x4_RENDA_FAMILIAR: De 6 a 10 salários mínimos (R$ 5.724,01 a R$ 9.540,00)', 'x5_ESCOLARIDADE_PAI: Ensino superior', 'x7_COMPANHIA_RESIDENCIA: Em casa ou apartamento, com cônjuge e/ou filhos', 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola privada', 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola pública', 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou parte no exterior', 'x9_MODALIDADE_ENSINO_MEDIO: Educação de Jovens e Adultos (EJA) e/ou Supletivo', 'x9_MODALIDADE_EN

In [ ]:
best_collector

['x0_REGIAO_IES: Centro-Oeste',
 'x0_REGIAO_IES: Nordeste',
 'x0_REGIAO_IES: Norte',
 'x0_REGIAO_IES: Sudeste',
 'x0_REGIAO_IES: Sul',
 'x1_SEXO: Masculino',
 'x2_COR_RACA: Amarela',
 'x2_COR_RACA: Branca',
 'x3_ESTADO_CIVIL: Casado(a)',
 'x3_ESTADO_CIVIL: Solteiro(a)',
 'x4_RENDA_FAMILIAR: Acima de 30 salários mínimos (mais de R$ 28.620,00)',
 'x4_RENDA_FAMILIAR: Até 1,5 salário mínimo (até R$ 1.431,00)',
 'x4_RENDA_FAMILIAR: De 4,5 a 6 salários mínimos (R$ 4.293,01 a R$ 5.724,00)',
 'x4_RENDA_FAMILIAR: De 6 a 10 salários mínimos (R$ 5.724,01 a R$ 9.540,00)',
 'x5_ESCOLARIDADE_PAI: Ensino superior',
 'x7_COMPANHIA_RESIDENCIA: Em casa ou apartamento, com cônjuge e/ou filhos',
 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola privada',
 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou a maior parte em escola pública',
 'x8_TIPO_ESCOLA_ENSINO_MEDIO: Todo ou parte no exterior',
 'x9_MODALIDADE_ENSINO_MEDIO: Educação de Jovens e Adultos (EJA) e/ou Supletivo',
 'x9_MODALIDADE_ENSINO_MEDIO

In [ ]:
X_train1_best = X_train[best_collector]
clf_best = model.fit(X_train1_best,y_train)
X_test1 = X_test[best_collector]
y_pred = clf_best.predict(X_test1)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.7976060935799782
